# TAL aplicado al análisis del discurso de los medios de prensa 📰🤓🔥


El proyecto consiste en entrenar y evaluar varios modelos de clasificación supervisada capaz de clasificar una noticia.


- Hito Unidad 1 (29 de septiembre): Datasets de entrenamiento y test + primer modelo baseline



### índex

1. [Importación del dataset](a)
2. [Extracción de tópicos vía RE en URLs](b)
3. [Extracción de términos clave de columnas title+text](b)
4. [Clasificación del dataset de entrenamiento](c)

## Generando dataset de entrenamiento
El dataset se clasificará en función de las siguientes etiquetas:
- mundo = 0 
- economía = 1 
- política y conflictos = 2 
- ciencia y tecnología = 3
- catástrofes y accidentes = 4 
- cultura y arte = 5 
- deporte = 6 
- ecología y planeta = 7
- crimen, delitos y justicia = 8 
- salud = 9

In [1]:
import warnings
warnings.filterwarnings('ignore')

# Data manipulation
import re
import multiprocessing
import pandas as pd
pd.set_option("display.max_rows", None)
import numpy as np

# NLP
import gensim
import gensim.corpora as corpora
from gensim.utils import simple_preprocess
from gensim.models import Word2Vec, CoherenceModel
#import pyLDAvis
#import pyLDAvis.gensim_models  # don't skip this
import nltk 
#nltk.download('stopwords') 
import spacy
from spacy.matcher import Matcher
nlp = spacy.load("es_core_news_sm")
print("versión de Spacy: ", spacy.__version__)



versión de Spacy:  3.1.3


### 1. Importación de dataset <a name="a"></a>

In [2]:
frames = [pd.read_csv("data/chile_2020-09.csv"), pd.read_csv("data/chile_2020-10.csv"),pd.read_csv("data/chile_2020-11.csv"), pd.read_csv("data/chile_2020-12.csv"), pd.read_csv("data/chile_2021-01.csv"), pd.read_csv("data/chile_2021-02.csv"), pd.read_csv("data/chile_2021-03.csv"), pd.read_csv("data/chile_2021-04.csv")]
dataset = pd.concat(frames)
dataset['content'] = dataset['title'] + " " + dataset['text'] 

print("Largo de dataset: ", len(dataset))
dataset= dataset[dataset['text'].notna()]
        
dataset.drop(columns=['country','year','date','id', 'id_journalist'], inplace=True)
dataset['topic'] = ""

Largo de dataset:  69702


### 2.  Extracción de tópicos vía RE en URL <a name="b"></a>

In [3]:
#identicando fuentes fuentes
set(dataset["media_outlet"])

{'ahoranoticiasmega',
 'biobiochile',
 'elciudadano',
 'elmostrador',
 'emol',
 'horas24',
 'latercera'}

In [4]:
print(dataset[dataset.media_outlet=="ahoranoticiasmega"]["url"].head(2))
print(dataset[dataset.media_outlet=="horas24"]["url"].head(2))
print(dataset[dataset.media_outlet=="elciudadano"]["url"].head(2))
print(dataset[dataset.media_outlet=="biobiochile"]["url"].head(2))
print(dataset[dataset.media_outlet=="elmostrador"]["url"].head(2))
print(dataset[dataset.media_outlet=="emol"]["url"].head(2))
print(dataset[dataset.media_outlet=="latercera"]["url"].head(2))

268    https://www.meganoticias.cl/nacional/312353-si...
269    https://www.meganoticias.cl/nacional/312350-cy...
Name: url, dtype: object
0    https://www.24horas.cl/coronavirus/tia-pikachu...
3    https://www.24horas.cl/nacional/rm-alcanza-sus...
Name: url, dtype: object
205    https://www.elciudadano.com/chile/el-avance-de...
206    https://www.elciudadano.com/latinoamerica/colo...
Name: url, dtype: object
29    https://www.biobiochile.cl/noticias/nacional/c...
30    https://www.biobiochile.cl/noticias/nacional/r...
Name: url, dtype: object
228    https://www.elmostrador.cl/noticias/pais/2020/...
230    https://www.elmostrador.cl/dia/2020/09/01/dipu...
Name: url, dtype: object
496    https://www.emol.com/noticias/Economia/2020/09...
497    https://www.emol.com/noticias/Economia/2020/09...
Name: url, dtype: object
209    https://www.latercera.com/politica/noticia/el-...
210    https://www.latercera.com/politica/noticia/cam...
Name: url, dtype: object


Las urls de _Ahoranoticiasmega, Horas24 y El Ciudadano_ son similares.
La url de _La Tercera_ posee dos tópicos, sin embargo, el segundo tópico es irrelevante.

In [5]:
#Generamos df1 concatenando ahoranoticiasmega, horas24 y elciudadano
medios = [dataset.loc[dataset['media_outlet']=="ahoranoticiasmega"],
          dataset.loc[dataset['media_outlet']=="horas24"],
          dataset.loc[dataset['media_outlet']=="elciudadano"],
          dataset.loc[dataset['media_outlet']=="latercera"]]
df1 = pd.concat(medios)
for index, row in df1.iterrows():
    url=row['url']
    obj = re.findall('(\w+)://([\w\-\.]+)/([\w\-]+).([\w\-]+)', url) 
    topic=obj[0][2]
    df1.loc[index,'topic'] = topic.lower()

Las urls de _Biobiochile, El Mostrador, Emol_ poseen dos columnas de tópicos. La primera columna entrega información irrelevante. 

In [6]:
medios2 = [dataset.loc[dataset['media_outlet']=="biobiochile"],
           dataset.loc[dataset['media_outlet']=="elmostrador"],
           dataset.loc[dataset['media_outlet']=="emol"]] 
df2 = pd.concat(medios2)
for index, row in df2.iterrows():
    url=row['url']
    obj = re.findall('(\w+)://([\w\-\.]+)/([\w\-]+).([\w\-]+)', url)
    topic = obj[0][3]
    df2.loc[index,'topic'] = topic.lower()

In [7]:
df = pd.concat([df1,df2])
np.sort(df['topic'].unique())

array(['2020', '2021', 'actualidad', 'animal', 'artes', 'autos',
       'calidad-de-vida', 'cartas-ciudadanas', 'chile',
       'ciencia-tecnologia', 'ciudadanos-al-poder', 'columnas',
       'conversacioneslt', 'coronavirus', 'culto', 'data', 'deportes',
       'derechos-humanos', 'destacados-cultura', 'economia', 'educacion',
       'el-deportivo', 'especiales', 'espectaculos', 'genero',
       'internacional', 'justicia', 'la-tercera-domingo', 'la-tercera-pm',
       'la-tercera-tv', 'latinoamerica', 'mapuche', 'masdeco',
       'medio-ambiente', 'mexico', 'mineria', 'mouse', 'multimedia',
       'mundo', 'nacional', 'noticiasbbc', 'opinion', 'pais', 'paula',
       'peru', 'plebiscito-chile-elige', 'politica', 'portada',
       'portada-tv', 'proceso-constituyente', 'programas', 'pueblos',
       'pulso', 'pulso-trader', 'que-pasa', 'reconstitucion',
       'redes-sociales', 'regiones', 'reportaje-investigacion',
       'revista-que-pasa', 'salud', 'sociedad', 'tecnologia',
       

In [8]:
df['label'] = ""
# mundo
df.loc[df['topic'] == 'mundo',                  'label'] = 0
df.loc[df['topic'] == 'noticiasbbc',            'label'] = 0
df.loc[df['topic'] == 'internacional',          'label'] = 0
# economia
df.loc[df['topic'] == 'economia',               'label'] = 1
df.loc[df['topic'] == 'pulso-trader',           'label'] = 1
# política y conflictos
df.loc[df['topic'] == 'politica',               'label'] = 2
df.loc[df['topic'] == 'plebiscito-chile-elige', 'label'] = 2
df.loc[df['topic'] == 'ciudadanos-al-poder',    'label'] = 2
df.loc[df['topic'] == 'proceso-constituyente',  'label'] = 2
# ciencia y tecnologia
df.loc[df['topic'] == 'tecnologia',             'label'] = 3
df.loc[df['topic'] == 'tecnologia-2',           'label'] = 3
df.loc[df['topic'] == 'ciencia-tecnologia',     'label'] = 3
df.loc[df['topic'] == 'mouse',                  'label'] = 3
df.loc[df['topic'] == 'redes-sociales',         'label'] = 3
# accidentes 
df.loc[df['topic'] == 'accidentes',             'label'] = 4
# arte y cultura
df.loc[df['topic'] == 'cultura',                'label'] = 5
df.loc[df['topic'] == 'artes',                  'label'] = 5
# deportes
df.loc[df['topic'] == 'deportes',               'label'] = 6
df.loc[df['topic'] == 'el-deportivo',           'label'] = 6
# ecologia y planeta
df.loc[df['topic'] == 'medio-ambiente',         'label'] = 7
df.loc[df['topic'] == 'animal',                 'label'] = 7
# crimen, delitos y justicia
df.loc[df['topic'] == 'justicia',               'label'] = 8
df.loc[df['topic'] == 'reportaje-investigacion','label'] = 8
# salud
df.loc[df['topic'] == 'salud',                  'label'] = 9
df.loc[df['topic'] == 'calidad-de-vida',        'label'] = 9
df.loc[df['topic'] == 'coronavirus',            'label'] = 9

display(df['label'].value_counts())
# nos quedamos sólo con las noticias etiquetadas. para clasificar df1 utilizaremos otro metodo
df1 = df.loc[df['label']==''] 
df = df[df['label']!='']

     34315
0     9913
6     9609
3     5689
2     5020
1     1774
8      428
9      244
5      179
7      168
Name: label, dtype: int64

Tenemos muchas noticias no clasificadas, por lo que es necesario utilizar otro mecanísmo para clasificar las noticias restantes.

### 3.  Extracción de términos clave de columna title+text <a name="b"></a>

In [ ]:
matcher = Matcher(nlp.vocab)

# Pattern 1: NOUN de NOUN
pattern_1 = [{"POS": "NOUN"},{"LOWER": "de"}, {"POS": "NOUN"}]
matcher.add("NOUN-de-NOUN", [pattern_1])

# Pattern 2: NOUN ADJ
pattern_2 = [{"POS": "NOUN"}, {"POS": "ADJ"}]
matcher.add("NOUN-ADJ", [pattern_2])

# Pattern 3: 
pattern_3 = [{"POS": "NOUN"}]
matcher.add("NOUN", [pattern_3])

    Bloque de alto coste computacional ⬇️

In [ ]:
df1['terms'] = ""
for index, row in df1.iterrows():
    doc = nlp(row['content'].lower())

    matches = matcher(doc)
    categories = ""
    for match_id, start, end in matches:
        string_id = nlp.vocab.strings[match_id]  # Get string representation
        span = doc[start:end]  # The matched span
        categories = categories + span.text + "; "
    
    row['terms'] = categories
    
display(df1.head(3))

In [ ]:
from collections import Counter
df1["most_common"] = ""
df1["second_common"] = ""

for index, row in df1.iterrows():
    split_it = row['terms'].split("; ")
    counter = Counter(split_it)
    row['most_common'] = counter.most_common()[0][0]
    try:
        row['second_common'] = counter.most_common()[1][0]
    except IndexError:
        row['second_common'] = counter.most_common()[0][0]

display(df1.head(3))

**Para no ejecutar siempre la celda de alto costo computacional, almacenamos la data procesada en un nuevo csv con los términos claves y luego solo importamos el csv**

In [ ]:
df1.to_csv("keysTerms.csv", index=False) 

In [9]:
df2=pd.read_csv("keysTerms.csv")
df2.replace(np.nan,"", inplace=True)

Etiquetamos los términos más comunes que pertenezcan a una de las categorías

In [10]:
df2.loc[df2['most_common'] == 'trump',                  'label'] = 0
df2.loc[df2['most_common'] == 'putin',                  'label'] = 0
df2.loc[df2['most_common'] == 'mundo',                  'label'] = 0
df2.loc[df2['most_common'] == 'dólar',                  'label'] = 1
df2.loc[df2['most_common'] == 'economía',               'label'] = 1
df2.loc[df2['most_common'] == 'retiro',               'label'] = 1
df2.loc[df2['most_common'] == 'presidente',             'label'] = 2
df2.loc[df2['most_common'] == 'ministro',               'label'] = 2
df2.loc[df2['most_common'] == 'diputado',               'label'] = 2
df2.loc[df2['most_common'] == 'elecciones',               'label'] = 2
df2.loc[df2['most_common'] == 'plebiscito',               'label'] = 2
df2.loc[df2['most_common'] == 'diputados',              'label'] = 2
df2.loc[df2['most_common'] == 'diputado',              'label'] = 2
df2.loc[df2['most_common'] == 'gobierno',               'label'] = 2
df2.loc[df2['most_common'] == 'constitución',           'label'] = 2
df2.loc[df2['most_common'] == 'senador',                'label'] = 2
df2.loc[df2['most_common'] == 'convención',             'label'] = 2
df2.loc[df2['most_common'] == 'servel',                 'label'] = 2
df2.loc[df2['most_common'] == 'votos',                 'label'] = 2
df2.loc[df2['most_common'] == 'primarias',                 'label'] = 2
df2.loc[df2['most_common'] == 'candidato',                 'label'] = 2
df2.loc[df2['most_common'] == 'primarias',                 'label'] = 2
df2.loc[df2['most_common'] == 'accidente',                 'label'] = 4
df2.loc[df2['most_common'] == 'incendio',                 'label'] = 4
df2.loc[df2['most_common'] == 'sismo',                 'label'] = 4
df2.loc[df2['most_common'] == 'grammy',                 'label'] = 5
df2.loc[df2['most_common'] == 'película',                 'label'] = 5 #no estoy seguro
df2.loc[df2['most_common'] == 'actor',                 'label'] = 5 #no estoy seguro
df2.loc[df2['most_common'] == 'cantante',                 'label'] = 5 #no estoy seguro
df2.loc[df2['most_common'] == 'canción',                 'label'] = 5 #no estoy seguro
df2.loc[df2['most_common'] == 'monumento',                 'label'] = 5 #no estoy seguro
df2.loc[df2['most_common'] == 'evento',                 'label'] = 5 #no estoy seguro
df2.loc[df2['most_common'] == 'gol',                'label'] = 6
df2.loc[df2['most_common'] == 'fútbol',                'label'] = 6
df2.loc[df2['most_common'] == 'bosques',                'label'] = 7 #quizas no vaya (puede estar relacionado a noticias de quema de bosques)
df2.loc[df2['most_common'] == 'agua',                'label'] = 7
df2.loc[df2['most_common'] == 'eclipse',                'label'] = 7
df2.loc[df2['most_common'] == 'mar',                'label'] = 7
df2.loc[df2['most_common'] == 'violencia',              'label'] = 8
df2.loc[df2['most_common'] == 'carabineros',            'label'] = 8
df2.loc[df2['most_common'] == 'camioneros',             'label'] = 8
df2.loc[df2['most_common'] == 'acusación',              'label'] = 8
df2.loc[df2['most_common'] == 'fiscal',                 'label'] = 8
df2.loc[df2['most_common'] == 'robo',                   'label'] = 8
df2.loc[df2['most_common'] == 'ley',                    'label'] = 8
df2.loc[df2['most_common'] == 'prisión',                    'label'] = 8
df2.loc[df2['most_common'] == 'imputado',               'label'] = 8
df2.loc[df2['most_common'] == 'homicidio',              'label'] = 8
df2.loc[df2['most_common'] == 'ataque',              'label'] = 8
df2.loc[df2['most_common'] == 'droga',                  'label'] = 8
df2.loc[df2['most_common'] == 'femicidio',              'label'] = 8
df2.loc[df2['most_common'] == 'amenazas',              'label'] = 8
df2.loc[df2['most_common'] == 'salud',                  'label'] = 9
df2.loc[df2['most_common'] == 'dosis',                  'label'] = 9
df2.loc[df2['most_common'] == 'vacuna',                 'label'] = 9
df2.loc[df2['most_common'] == 'pacientes',                 'label'] = 9
df2.loc[df2['most_common'] == 'vacunas',                 'label'] = 9
df2.loc[df2['most_common'] == 'casos',                  'label'] = 9
df2.loc[df2['most_common'] == 'cáncer',                 'label'] = 9
df2.loc[df2['most_common'] == 'virus',                 'label'] = 9
df2.loc[df2['most_common'] == 'contagios',                 'label'] = 9
df2.loc[df2['second_common'] == 'gobierno'              'label'] = 2
df2.loc[df2['second_common'] == 'paro'                  'label'] = 2
df2.loc[df2['second_common'] == 'presidente'                 'label'] = 2
df2.loc[df2['second_common'] == 'elecciones'                 'label'] = 2
df2.loc[df2['second_common'] == 'primarias'                 'label'] = 2
df2.loc[df2['second_common'] == 'oposición'                 'label'] = 2
df2.loc[df2['second_common'] == 'fondos'                 'label'] = 1
df2.loc[df2['second_common'] == 'incendio'                 'label'] = 4
df2.loc[df2['second_common'] == 'sismo'                 'label'] = 4
df2.loc[df2['second_common'] == 'accidente'                 'label'] = 4
df2.loc[df2['second_common'] == 'actriz'                 'label'] = 5
df2.loc[df2['second_common'] == 'eclipse'                 'label'] = 7
df2.loc[df2['second_common'] == 'violencia'                 'label'] = 8
df2.loc[df2['second_common'] == 'prisión'                 'label'] = 8
df2.loc[df2['second_common'] == 'droga'                 'label'] = 8
df2.loc[df2['second_common'] == 'prisión preventiva'                 'label'] = 8
df2.loc[df2['second_common'] == 'salud'                 'label'] = 9
df2.loc[df2['second_common'] == 'vacuna'                 'label'] = 9
df2.loc[df2['second_common'] == 'seremi'                 'label'] = 9

# agregar mas...


In [11]:
df = pd.concat([df,df2])
df.drop(columns=['most_common','second_common','terms'], inplace=True)
df['label'].value_counts()

     27954
0     9957
6     9705
2     6422
3     5689
9     2249
1     2138
8     2037
4      416
7      394
5      378
Name: label, dtype: int64

### 4. Identificar subtópicos relacionados a los tópicos faltantes.

Para obtener un dataset balanceado, falta identificar noticias de:
* catástrofes y accidentes (4)
* ecología y planeta (7)
* cultura y arte (5)
* economía (1)

El título de una noticia representa información valiosa para conocer de que trata la noticia, por lo tanto, asociamos una palabra específica dentro del título con un tópico específico. Por ejemplo la palabra temblor, se asocia directamente con el tópico catástrofes y accidentes (4)

In [12]:
df2[df2['title'].str.contains("(?i)temblor")]['title']

0        Temblor de menor intensidad afecta nuevamente ...
8        "Con mucho susto": Alcalde de Huasco por segui...
10       Fuerte temblor de 6,3 grados afecta a la regió...
12                     Nuevo temblor se registra en Huasco
14             Temblor se registra en la región de Atacama
32       SHOA descarta riesgo de tsunami tras nuevo tem...
33       Imágenes revelan el daño que provocaron los te...
35       Onemi llama a la tranquilidad luego de los tem...
36       Nuevo temblor se registra en la zona norte del...
37       Revisa los videos de los temblores que afectar...
38       Temblor magnitud 7.0 afectó a la zona norte de...
63        Temblor afecta nuevamente a la Región de Atacama
64       De nube tóxica a fenómeno por temblor en el no...
101                    Nuevo temblor se registra en Huasco
104                  Temblor afecta a la región de Atacama
110      Nuevo temblor se registra en la región de Atacama
246                 Temblor afecta a la región de Coquim

In [13]:
df2 = df.loc[df['label']=='']
df = df[df['label']!='']

df2.loc[df2['title'].str.contains("(?i)temblor|erupción|incendio|catástrofe|accidente|herido|muert"), 'label'] = 4 #muert/o/e/os/es
df2.loc[df2['title'].str.contains("(?i)ecología|planeta|contaminación|atmósfera|eclipse|tierra|reciclaje|vegetación|paisaje|animal|verdes"), 'label'] = 7
df2.loc[df2['title'].str.contains("(?i)cultura|pintura|museo|música"), 'label'] = 5 #arte no por que toma otras palabras como valuarte, etc

df2.loc[df2['title'].str.contains("(?i)economía|bolsa|dinero"), 'label'] = 1 
df2.loc[df2['title'].str.contains("(?i)crimen|delito|asesin|justicia"), 'label'] = 8 
df2.loc[df2['title'].str.contains("(?i)salud|medicamentos"), 'label'] = 9 

df2.dropna(subset = ["label"], inplace=True)
df2['label'].value_counts()

     24876
4     1399
8      703
9      449
7      249
1      141
5      137
Name: label, dtype: int64

In [14]:
df = pd.concat([df,df2])
df['label'].value_counts()

     24876
0     9957
6     9705
2     6422
3     5689
8     2740
9     2698
1     2279
4     1815
7      643
5      515
Name: label, dtype: int64

In [15]:
df.replace("", np.nan, inplace=True)
df.dropna(subset = ["label"], inplace=True)

df = df.astype({"label": str})
df.replace("0.0","mundo", inplace=True)
df.replace("1.0","economía", inplace=True)
df.replace("2.0","política y conflictos", inplace=True)
df.replace("3.0","ciencia y tecnología", inplace=True)
df.replace("4.0","catástrofes y accidentes", inplace=True)
df.replace("5.0","cultura y arte", inplace=True)
df.replace("6.0","deporte", inplace=True)
df.replace("7.0","ecología y planeta", inplace=True)
df.replace("8.0","crimen, delitos y justicia", inplace=True)
df.replace("9.0","salud", inplace=True)

In [16]:
df.to_csv("dataEtiquetada.csv", index=False) 
df['label'].value_counts()

mundo                         9957
deporte                       9705
política y conflictos         6422
ciencia y tecnología          5689
crimen, delitos y justicia    2740
salud                         2698
economía                      2279
catástrofes y accidentes      1815
ecología y planeta             643
cultura y arte                 515
Name: label, dtype: int64